# Day 10: Syntax scoring
from [AdventOfCode website](https://adventofcode.com/2021])

You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:

Syntax error in navigation subsystem on line: all of them
All of them?! The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).

The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

- If a chunk opens with (, it must close with ).
- If a chunk opens with [, it must close with ].
- If a chunk opens with {, it must close with }.
- If a chunk opens with <, it must close with >.

So, () is a legal chunk that contains no other chunks, as is []. More complex but valid chunks include `([])`, `{()()()}`, `<([{}])>`, `[<>({}){}[([])<>]]`, and even `(((((((((())))))))))`.

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include `(]`, `{()()()>`, `(((()))}`, and `<([]){()}[{}])`. Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

For example, consider the following navigation subsystem:

```
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
```

Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

```
{([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
[[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
[{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
[<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
<{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.
```

Stop at the first incorrect closing character on each corrupted line.

Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the first illegal character on the line and look it up in the following table:

- ): 3 points.
- ]: 57 points.
- }: 1197 points.
- >: 25137 points.

In the above example, an illegal ) was found twice (2*3 = 6 points), an illegal ] was found once (57 points), an illegal } was found once (1197 points), and an illegal > was found once (25137 points). So, the total syntax error score for this file is 6+57+1197+25137 = 26397 points!

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

## Some considerations for this one

One stack shall do it, push-down automaton style.

- Push when open
- Pop when closed. If trying to pop when empty or not equal to the one top --> corrupt
- When this was done, we might have incomplete lines (that must contain only opening chars) --> incomplete
- If everything succedded --> complete

In [89]:
from functools import reduce
from collections import Counter

with open('input_day10.txt') as f:
    out = [x.strip() for x in f.readlines()]

from enum import Enum, auto

class Status(Enum):
    CORRUPT = auto()
    INCOMPLETE = auto()
    COMPLETE = auto

# Some helper function for debugging
def log(tuple):
    if tuple[0] == Status.COMPLETE:
        print('Line was complete and correct')
    if tuple[0] == Status.INCOMPLETE:
        print(f'Line was incomplete, the line is {tuple[3]}')
    if tuple[0] == Status.CORRUPT:
        print(f'Got {tuple[2]} but expected {tuple[1]}') 

def isCorrupt(line):
    from queue import LifoQueue
    stack = LifoQueue(1000)

    for p in line:
        if p == '[' or p == '(' or p == '{' or p == '<':            
            stack.put(p)
        else:
            c = stack.get()            
            if p == ']':
                if c != '[':                    
                    return (Status.CORRUPT, c, p)
            if p == '}':
                if c != '{':
                    return (Status.CORRUPT, c, p)
            if p == ')':
                if c != '(':
                    return (Status.CORRUPT, c, p)
            if p == '>':
                if c != '<':
                    return (Status.CORRUPT, c, p)

    if stack.empty():
        return (Status.COMPLETE, None, None)

    # Check if incomplete    
    while stack.qsize() != 0:
        c = stack.get()
        if(c != '[' and c != '(' and c != '<' and c != '{'):
            return (Status.CORRUPT, c)
    
    return (Status.INCOMPLETE, None, None, line)

def score(res):
    v = res[2] # Where the gotten value is
    if v == ')': return 3
    if v == ']': return 57
    if v == '}': return 1197
    if v == '>': return 25137

filtered = [isCorrupt(x) for x in out]
wrongOnes = list(filter(lambda x: (x[0] == Status.CORRUPT), filtered))
score = sum(list(map(score, wrongOnes)))
print(score)

319233


Now, discard the corrupted lines. The remaining lines are incomplete.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out the sequence of closing characters that complete all open chunks in the line.

You can only use closing characters (), ], }, or >), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

In the example above, there are five incomplete lines:

```
[({(<(())[]>[[{[]{<()<>> - Complete by adding }}]])})].
[(()[<>])]({[<{<<[]>>( - Complete by adding )}>]}).
(((({<>}<{<{<>}{[]{[]{} - Complete by adding }}>}>)))).
{<[[]]>}<{[{[{[]{()[[[] - Complete by adding ]]}}]}]}>.
<{([{{}}[<[[[<>{}]]]>[]] - Complete by adding ])}>.
```

Did you know that autocomplete tools also have contests? It's true! The score is determined by considering the completion string character-by-character. Start with a total score of 0. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

- ): 1 point.
- ]: 2 points.
- }: 3 points.
- >: 4 points.

So, the last completion string above - ])}> - would be scored as follows:

- Start with a total score of 0.
- Multiply the total score by 5 to get 0, then add the value of ] (2) to get a new total score of 2.
- Multiply the total score by 5 to get 10, then add the value of ) (1) to get a new total score of 11.
- Multiply the total score by 5 to get 55, then add the value of } (3) to get a new total score of 58.
- Multiply the total score by 5 to get 290, then add the value of > (4) to get a new total score of 294.

The five lines' completion strings have total scores as follows:

```
}}]])})] - 288957 total points.
)}>]}) - 5566 total points.
}}>}>)))) - 1480781 total points.
]]}}]}]}> - 995444 total points.
])}> - 294 total points.
```

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is 288957 because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

# What was done
Here we take the same approach as in part 1 but as we know that we only have incomplete lines, the push-down automata logics is simplified. We shall not forget to remove the working pairs though!

Computing the score is simply the application of some formulas, nothing special or beautiful here.

In [92]:
# Get the elements to be added to complete the stack
def complete(incomplete):
    from queue import LifoQueue
    stack = LifoQueue(1000)

    # Rebuild a stack and remove the pairs that are complete
    for p in incomplete:
        if p == '[' or p == '(' or p == '{' or p == '<':            
            stack.put(p)
        else:
            c = stack.get()

    missing = []

    # Rebuild the missing stack symbols
    while stack.qsize() != 0:
        c = stack.get()
        missing.append(c)

    # Replace the letters by their closing counterpart
    missing = "".join(missing)
    replace = {'(': ')', '<': '>', '{': '}', '[': ']'}

    for x,y in replace.items():
        missing = missing.replace(x,y)

    return missing

def score2(list):
    s = 0    
    for elem in list:
        val = 1
        if elem == ']':
            val = 2
        if elem == '}':
            val = 3
        if elem == '>':
            val = 4
        
        s *= 5
        s += val
    return s

# Take the incompletes values from part 1, which have the original line stored as well
incompletes = list(map(lambda x: x[3], list(filter(lambda x: (x[0] == Status.INCOMPLETE), filtered))))

completions = [complete(x) for x in incompletes]
scores = [score2(x) for x in completions]
scores.sort()
print(scores[len(scores) // 2])

1118976874
